# Estimate power-law creep envelopes

Copyright 2020 Marco A. Lopez-Sanchez.  
Content under [Creative Commons Attribution license CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/), code under [Mozilla Public License 2.0](https://www.mozilla.org/en-US/MPL/2.0/).

> **Goal**: Understand flow laws and estimate power-law creep envelopes for different rock types.

## Introduction

Below depths of about 10-15 km, the temperature is high enough to favour creep processes, a phenomenon by which the material is permanently deformed under constant stress without losing coherence, over those of friction. To describe and predict how rocks deform under such conditions, we need a mathematical rule that relates stress (or force) to strain or strain rate. We called such relation a **flow law**, _deformation law_, or a *constitutive equation*. The adjective _constitutive_ means that this relation depends on the constitution of the material. Many polycrystalline solid materials, such as rocks, glaciers or metals show that at temperatures above half the melting temperature $>0.5T_m$ the strain rate $\dot{\epsilon}$ is proportional to the differential stress raised to some power of n ($\dot{\epsilon} \propto \sigma^n_d$) as long as the shear stress is above a threshold value called the [yield stress](https://en.wikipedia.org/wiki/Yield_(engineering)). Below that limit, however, an elastic (recoverable) deformation occurs. This behaviour is generally referred to as [plasticity](https://en.wikipedia.org/wiki/Plasticity_(physics)) or plastic deformation. The two most important features of plastic deformation is that **once the yield stress is exceeded**, the final deformation depends mainly on time and temperature (which strongly affects strain rates). It is therefore said that plasticity is a process with a strong dependence on time and temperature.

## Step 1: The power-law creep approximation

At low to moderate (<200 MPa) stresses, those expected to operate in the ductile lithosphere, the relationship between differential stress and strain rate for most of the rocks in the laboratory fits a relationship referred to as the _power-law creep_ of the form

$$
\dot{\epsilon} = \sigma^n_d A \space exp \left( \frac{E}{RT} \right)
$$
  
$$
\sigma_d = \dot{\epsilon}^{(1/n)} \left( \frac{1}{A} \right)^{1/n} \space exp \left( \frac{E}{nRT} \right)
$$

where $\sigma_d$ is the differential stress in _MPa_ and $n$ the power-law exponent, $\dot{\epsilon}$ is the strain rate in $s^{-1}$ wich considers the _time dependence_ of the deformation, and $A \space exp(E/RT)$ is the Arrhenius equation (see box below for details) that considers the _temperature dependence_ of the deformation, the higher the temperature the higher the strain rate for a similar differential stress.

> The [Arrhenius equation](https://en.wikipedia.org/wiki/Arrhenius_equation) is a formula proposed by [Svante Arrhenius](https://en.wikipedia.org/wiki/Svante_Arrhenius) to model reaction rates, i.e. the frequency of events per second that result in a reaction. This equation, however, is used in a general way for any process involving rates and a dependence on temperature such as for example viscosity in fluids or plasticity in solids. In our case, it will account for the frequency of events that lead to a permanent crystal lattice deformation. The equation consists of two main parts, the _pre-exponential factor_ $(A)$ which in a chemical reaction accounts for the number of events per second leading to or not to a reaction (or a permanent crystal lattice deformation), and the _exponential factor_ $[exp(E/RT)]$ that accounts for the probability that any given event will result in a reaction (or a permanent crystal lattice deformation). The _exponential factor_ includes the _activation energy_ $(E)$ required for the reaction or any other process to be modelled in $J mol^{-1}$, the _universal gas constant_ $(R)$ in $J mol^{-1} K^{-1}$, and the _absolute temperature_ $(T)$ in kelvins. The _pre-exponential factor_ $(A)$ and the _activation energy_ $(E)$ are both material-specific variables and are estimated in the laboratory for different types of rock.

At moderate differential stresses (20-200 MPa) and temperatures above half the melting temperature $>0.5T_m$, most rock types in the laboratory usually provide values of $n$ between 3 and 5. At these conditions, rock samples typically develop dynamic recrystallization and crystallographic preferred orientation (CPO), both typical features of a deformation controlled by the movement of dislocations known as [dislocation creep](https://en.wikipedia.org/wiki/Dislocation_creep). This observation is in excellent agreement with what is observed in nature, where the most typical feature in natural ductile shear zones below 10-15 km is the presence of fine-grained rocks known as [mylonites](https://en.wikipedia.org/wiki/Mylonite) characterized by these very same microstructures. Hence, geophysicists usually use power-law equations with $n$ values typical of the dislocation creep to model the deformation of the non-frictional part of the lithosphere, a fairly reliable first approximation since most of the deformation in the lithosphere is accommodated through [crustal-scale shear zones](https://doi.org/10.1016/j.tecto.2012.06.006).


### The effect of pressure, grain size, and water fugacity

TODO 


we finally obtain a power-law relation like this

$$
\dot{\epsilon} = \sigma^n_d \space d^{-m} \space f_{H_2O}^r \space A \space exp \left( \frac{E + PV}{RT} \right)
$$

where $d$ is the average grain size in μm and $m$ the grain size exponent, and $f_{H_2O}$ and $r$ are the water fugacity (water molecules per 1e6 Si atoms) and the water fugacity exponent, respectively. Note that if $m$ and $r$ equals zero, the effect of grain size and/or water fugacity is ignored.

In [1]:
def power_law_creep(srate, A, n, exp_fact, gsize=(1, 0), water=(1,0)):
    """ Return the neccesary differential stress (Tresca criterion) in
    MPa for permanently deforming a polycrystalline material at a given
    environmental conditions.

    Parameters (scalars and tuples, all positive values)
    ----------
    srate : strain rate [s**-1]
    n : stress exponent
    A : material constant [MPa**-n s**-1]
    exp_fact : tuple of dim 4 (E, T, P, V)
        E : activation energy [J mol**-1]
        T : absolute temperature [K]
        P : pressure [MPa]
        V : activation volume per mol [m**3 mol**-1]
    gsize : optional, tuple of dim 2 (d, m); (1, 0) by default
        d : average grain size [microns]
        m : grain size exponent [in absolute value]
    water : optional, tuple of dim 2 (f, r); (1, 0) by default
        f : fugacity of water [water molecules per 1e6 Si atoms]
        r : water fugacity exponent

    Assumptions
    -----------
    - Steady-state creep
    - Low to moderate stress regime (< 200 MPa)
    - Effect of partial melt ignored
    """
    # set universal gas constant [J mol**-1 K**-1]
    R = 8.3144626 
    
    # extract parameters
    E, T, P, V = exp_fact
    d, m = gsize
    f, r = water

    return (ss * (d**-m) * (f**r) * np.exp((E + P * V) / (R * T)) / A)**(1 / n)